In [ ]:
# perceptron simple para función OR y AND
def salida(k, pesos, t):
  z=-t
  for i in range(len(k)):
    z=z+(k[i]*pesos[i])
  if z>=0:
    return 1
  else:
    return 0


def entrenar_perceptron(datos_ent, pesos, t, l):
  errores=True
  while errores:
    errores=False
    # entrenar perceptron
    for k,y in datos_ent.items():
      z=salida(k, pesos, t)
      if z!=y:
        errores=True
        # error
        e=(y-z)
        # calcular ajustes
        delta_t=-(l*e)
        t=t+delta_t
        for i in range(len(k)):
          delta_w=l*e*k[i]
          pesos[i]=pesos[i]+delta_w

  return pesos, t


def clasificar(entrada, pesos, t):
  return salida(entrada, pesos, t)


if __name__ == "__main__":
  Función=0
  Función= input("Función lógica deseada OR (1), AND (2): ")
  Función= int(Función)
  if Función == 1:
    datos_ent={(0,0):0, (0,1):1, (1,0):1,(1,1):1}
    pesos=[0.2,0.6]
    t=0.4
    l=0.2
    pesos, t = entrenar_perceptron(datos_ent, pesos, t, l)
  else:
    datos_ent={(0,0):0, (0,1):0, (1,0):0,(1,1):1}
    pesos=[0.2,0.6]
    t=0.4
    l=0.2
    pesos, t = entrenar_perceptron(datos_ent, pesos, t, l)

  X1= input("Ingrese el valor para X1:")
  X1= int(X1)
  X2= input ("Ingrese el valor para X2:")
  X2= int(X2)
  print (clasificar((X1,X2), pesos, t))

Función lógica deseada OR (1), AND (2): 1
Ingrese el valor para X1:1
Ingrese el valor para X2:1
1


In [ ]:
# Perceptrón multicapa
# para calculo de función XOR
import math
import random
import string

# crea una matriz para almacenar los pesos
def matriz(x, y):
  m = []
  for i in range(x):
      m.append([0.0]*y)
  return m

# función de tipo sigmoide
def sigmoide(x):
  return math.tanh(x)

# derivada de función de tipo sigmoide
def dsigmoide(x):
  return 1.0 - x**2

# inicialización
def iniciar_perceptron():
  global nodos_ent, nodos_ocu, nodos_sal, pesos_ent, pesos_sal
  global act_ent, act_ocu, act_sal
  random.seed(0)

  # sumamos uno para umbral en nodos entrada
  nodos_ent = nodos_ent + 1

  # activación de los nodos
  act_ent = [1.0]*nodos_ent
  act_ocu = [1.0]*nodos_ocu
  act_sal = [1.0]*nodos_sal

  # crear matrices de pesos
  pesos_ent = matriz(nodos_ent, nodos_ocu)
  pesos_sal = matriz(nodos_ocu, nodos_sal)
  # inicializar pesos a valores aleatorios
  for i in range(nodos_ent):
      for j in range(nodos_ocu):
          pesos_ent[i][j] = random.uniform(-0.5, 0.5)
  for j in range(nodos_ocu):
      for k in range(nodos_sal):
          pesos_sal[j][k] = random.uniform(-0.5, 0.5)


# actualizar valor de los nodos
def actualiza_nodos(entradas):
  global nodos_ent, nodos_ocu, nodos_sal, pesos_ent, pesos_sal
  global act_ent, act_ocu, act_sal
  if len(entradas) != nodos_ent-1:
      raise ValueError('Numero de nodos de entrada incorrectos')

  # activación en nodos de entrada
  for i in range(nodos_ent-1):
      act_ent[i] = entradas[i]

  # activación en nodos ocultos
  for j in range(nodos_ocu):
      sum = 0.0
      for i in range(nodos_ent):
          sum = sum + act_ent[i] * pesos_ent[i][j]
      act_ocu[j] = sigmoide(sum)

  # activación en nodos de salida
  for k in range(nodos_sal):
      sum = 0.0
      for j in range(nodos_ocu):
          sum = sum + act_ocu[j] * pesos_sal[j][k]
      act_sal[k] = sigmoide(sum)

  return act_sal[:]


# retropropagación de errores
def retropropagacion(objetivo, l):
  global nodos_ent, nodos_ocu, nodos_sal, pesos_ent, pesos_sal
  global act_ent, act_ocu, act_sal
  if len(objetivo) != nodos_sal:
      raise ValueError('numero de objetivos incorrectos')

  # error en nodos de salida
  delta_salida = [0.0] * nodos_sal
  for k in range(nodos_sal):
      error = objetivo[k]-act_sal[k]
      delta_salida[k] = dsigmoide(act_sal[k]) * error

  # error en nodos ocultos
  delta_oculto = [0.0] * nodos_ocu
  for j in range(nodos_ocu):
      error = 0.0
      for k in range(nodos_sal):
          error = error + delta_salida[k]*pesos_sal[j][k]
      delta_oculto[j] = dsigmoide(act_ocu[j]) * error

  # actualizar pesos de nodos de salida
  for j in range(nodos_ocu):
      for k in range(nodos_sal):
          cambio = delta_salida[k]*act_ocu[j]
          pesos_sal[j][k] = pesos_sal[j][k] + l*cambio

  # actualizar pesos de nodos de entrada
  for i in range(nodos_ent):
      for j in range(nodos_ocu):
          cambio = delta_oculto[j]*act_ent[i]
          pesos_ent[i][j] = pesos_ent[i][j] + l*cambio

  # calcular error
  error = 0.0
  for k in range(len(objetivo)):
      error = error + 0.5*(objetivo[k]-act_sal[k])**2
  return error


# clasificar patrón de entrada
def clasificar(patron):
  for p in patron:
      salida = actualiza_nodos(p[0])
      salida1 = salida [0]
      if salida1 <= 0.05 :
          salida1 = 0
      else:
         salida1  = 1
      print (p[0], '->', salida1)


# entrenamiento del perceptrón
# max_iter=1000)
def entrenar_perceptron(patron, l):
  max_iter=1000
  for i in range(max_iter):
    error = 0.0
    for p in patron:
      entradas = p[0]
      objetivo = p[1]
      actualiza_nodos(entradas)
      error = error + retropropagacion(objetivo, l)
    # salir si alcanzamos el limite inferior de error deseado
    if error < 0.001:
      break



if __name__ == '__main__':
  datos_ent =[[[1,1,1,1,1,0,0,1,1,1,1,1,1,0,0,1,1,0,0,1], [1,0,0,0,0]],
  [[1,1,1,1,1,0,0,0,1,1,1,1,1,0,0,0,1,1,1,1], [0,1,0,0,0]],
  [[1,1,1,1,0,0,1,0,0,0,1,0,0,0,1,0,1,1,1,1], [0,0,1,0,0]],
  [[1,1,1,1,1,0,0,1,1,0,0,1,1,0,0,1,1,1,1,1], [0,0,0,1,0]],
  [[1,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1,1,1,1], [0,0,0,0,1]]]

  datos_entrenamientico =[[[0,1,1,1,1,0,0,1,1,1,1,1,1,0,0,1,1,0,0,1], [1,0,0,0,0]],
  [[1,1,1,1,1,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1], [0,1,0,0,0]],
  [[1,1,1,1,0,0,0,0,0,0,1,0,0,0,1,0,1,1,1,1], [0,0,1,0,0]],
  [[1,1,1,1,1,0,0,1,0,0,0,1,1,0,0,1,1,1,1,1], [0,0,0,1,0]],
  [[1,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1,1,1,0], [0,0,0,0,1]]]


  nodos_ent=20  # dos neuronas de entrada
  nodos_ocu=30  # dos neuronas ocultas
  nodos_sal=5 # una neurona de salida
  l=0.5
  iniciar_perceptron()
  entrenar_perceptron(datos_ent, l)
  clasificar(datos_entrenamientico)

[0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1] -> 1
[1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1] -> 1
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1] -> 1
[1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1] -> 1
[1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0] -> 1
